In this notebook, we will perform the project smile detection.

The basic idea is that we train a LetNet network by dataset SMILES which cantains 13,165 images of smile or not smile face. After training the model, we access our webcam/video file and apply smile detection to each frame. 
Faces are detected by Haar cascade face detector and then extracted from images to feed the LeNet network.


Note: there are 13,165 images in the dataset, 9,475 of these examples are not smiling while only 3,690 belong to the smiling class.

In [2]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from keras.utils import np_utils
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import imutils
import cv2
import os

Using TensorFlow backend.


In [2]:
data = []
labels = []

DATASET_PATH = "SMILEsmileD/SMILEs" 

for imagePath in sorted(list(paths.list_images(DATASET_PATH))):
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = imutils.resize(image, width=28)
    image = img_to_array(image)
    data.append(image)
    
    # extract the class label from the image path and update the labels list
    label = imagePath.split(os.path.sep)[-3]
    lable = "smiling" if label == "positives" else "not_smiling"
    labels.append(label)
"""
The SMILES dataset stores smiling faces in the  SMILES/positives/positives7 subdirectory 
while not smiling faces live in the SMILES/negatives/negatives7 subdirectory.

Therefore, given the path to an image:

    SMILEs/positives/positives7/10007.jpg
    
We can extract the class label by splitting on the image path separator 
and grabbing the third-to-last subdirectory: positives. 
"""    
#print(type(labels[1]))
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
#print(type(labels[1]))
#print(labels[:5])
le = LabelEncoder().fit(labels)
labels = np_utils.to_categorical(le.transform(labels), 2)
#print(type(labels))
#print(type(labels[1,1]))
#print(labels.shape)

As mentioned before, the data is unbalance, namley the number of negative images is much greater than that of positive. We could handle it by weighting.

In [3]:
#compute prorportions of each catogory
classTotals = labels.sum(axis=0)
classWeight = classTotals.max() / classTotals


"""
label contains two kinds of element, [1, 0] and [0, 1]. Thus, the number of each catogory 
can be count easily by sum.

The weight is approx. [1, 2.56] for [negative, positive]
This weighting implies that our network will treat 
every instance of “smiling” as 2.56 instances of “not smiling” 
and helps combat the class imbalance issue 
by amplifying the per-instance loss 
by a larger weight when seeing “smiling” examples.
"""

#split the whole data to train and test data
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.20, stratify=labels, random_state=42)

### Train LeNet

In [4]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Dropout
from keras import optimizers
from sklearn.metrics import classification_report

#The number of categroies of letter 
Num_classes = 2

print("[INFO] compiling model...")
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(Num_classes, activation='sigmoid')) #2 classes not need softmax

model.summary()



[INFO] compiling model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)             

2021-10-03 19:26:13.806280: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-03 19:26:13.806939: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


### Compile and train network

In [5]:
# print("[INFO] compiling model...")
# model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

# Epochs = 15
# print("[INFO] training network...")
# H = model.fit(trainX, trainY, validation_data=(testX, testY),
#               class_weight=classWeight, batch_size=64, epochs=Epochs, verbose=1)
# """
# class_weight is used for imbalance issue

# Explanation:
# https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras

# """

### Evaluation and save

In [6]:
# print("[INFO] evaluating network...")
# predictions = model.predict(testX, batch_size=64)
# print(classification_report(testY.argmax(axis=1),
#     predictions.argmax(axis=1), target_names=le.classes_))

# MODEL_FILENAME = "Smile_Detection.hdf5"
# print("[INFO] saving model...")
# model.save(MODEL_FILENAME)
# print("[INFO] finished")

### curve for perfomance

In [7]:
# plt.style.use("ggplot")
# plt.figure()
# plt.plot(np.arange(0, Epochs), H.history['loss'], label="train_loss")
# plt.plot(np.arange(0, Epochs), H.history['val_loss'], label="val_loss")
# plt.plot(np.arange(0, Epochs), H.history['accuracy'], label="acc")
# plt.plot(np.arange(0, Epochs), H.history['val_accuracy'], label="val_loss")
# plt.title("Training Loss and Accuracy")
# plt.xlabel("Epoch #")
# plt.ylabel("Loss/Accuracy")
# plt.legend()
# plt.show()

# """
# Note: H.history['accuracy'] or H.history['acc']
# denpends on model.compile(... ,metrics=["accuracy"]) or
# model.compile(... ,metrics=["acc"])
# """

### Running the Smile CNN in Real-time
We've trained our model, then we need access webcam/video file and detect smile

In [ ]:
CASCADE = "haarcascade_frontalface_default.xml"
MODEL = "Smile_Detection.hdf5"
VIDEO = "Smile_video.mp4"

detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
model = load_model(MODEL)
camera = cv2.VideoCapture(0) #from webcam
#camera = cv2.VideoCapture(VIDEO) #from video file

while True:
    # grab the current frame
    (grabbed, frame) = camera.read()
    
    # if we are viewing a video and we did not grab a frame, 
    #then we have reached the end of the video
    if VIDEO != "" and not grabbed:
        break
    # Otherwise    
    # resize the frame, convert it to grayscale, and then clone the
    # original frame so we can draw on it later in the program
    frame = imutils.resize(frame, width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frameClone = frame.copy()
    
    # detect faces in the input frame, then clone the frame 
    # so that we can draw on it
    rects = detector.detectMultiScale(gray, scaleFactor=1.1,
                                      minNeighbors=5, minSize=(30, 30),
                                      flags=cv2.CASCADE_SCALE_IMAGE)

    # loop over the face bounding boxes
    for (fX, fY, fW, fH) in rects:
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (28, 28))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        (notSmiling, smiling) = model.predict(roi)[0]
        label = "Smiling"  if smiling > notSmiling else "Not_Smiling"
        if smiling > notSmiling:
        # display the label and bounding box rectangle on the output # frame
            cv2.putText(frameClone, label, (fX, fY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                      (0, 0, 255), 2)
        else:
            cv2.putText(frameClone, label, (fX, fY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
            cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                      (0, 255, 0), 2)
        # show our detected faces along with smiling/not smiling labels
        cv2.imshow("Face", frameClone)
        
         # if the ’q’ key is pressed, stop the loop
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break




"""
The .detectMultiScale method handles detecting the bounding box 
(x,y)-coordinates of faces in the frame
    
It returns a list of 4-tuples that make up the rectangle 
that bounds the face in the frame. 
The first two values in this list are the starting (x, y)-coordinates. 
The second two values in the rects list are
the width and height of the bounding box, respectively.
"""

"""
Inside the for loop,
extract the ROI of the face from the grayscale image,
resize it to a fixed 28x28 pixels, and then prepare the
ROI for classification via the CNN
"""

`CascadeClassifier` function requires `opencv-contrib-python`. While we cannot install by conda directly. 

Hence, we need install `pip` in conda environment `keras_env`

So, firstly, activate the environment by
`conda activate keras_env`.

Then, go to the envrionment, the path can be found by
`which python`

It returns `/opt/anaconda3/envs/keras_env/bin/python`

Next, we can install `opencv-contrib-python` by `/opt/anaconda3/envs/keras_env/bin/pip install opencv-contrib-python`

After that, restart this notebook. Finally, everything is find.

In [ ]:
camera.release()
cv2.destroyAllWindows()